In [14]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from urllib.request import urlopen
import kglab
import pandas as pd
import seaborn as sns
from rdflib import Graph  #, plugin
import matplotlib.pyplot as plt
# pyshack sends output to log along with the vars.  This suppresses that
import logging, sys
logging.disable(sys.maxsize)
from reportlab.platypus import *
from datetime import datetime
from minio import Minio

In [15]:
namespaces = {
    "shacl":   "http://www.w3.org/ns/shacl#" ,
    }

kg = kglab.KnowledgeGraph(
    name = "Schema.org based datagraph",
    base_uri = "https://example.org/id/",
    namespaces = namespaces,
    )

In [16]:
date_time = datetime.now().date()

sgurl = "https://raw.githubusercontent.com/iodepo/odis-arch/master/validation/SHACL/GeoCodesv1Shapes.ttl"
sg = urlopen(sgurl)

# Create client with access and secret key.
client = Minio("minio server", "access key", "secret key")
bucket = "yybucket"

#path = f"milled/iris/001dcdf99c383571c0479c00def62ed953c83a92.rdf"
#response = client.get_object(bucket, path)
#rdf = response.data

#path = f"summoned/iris/001dcdf99c383571c0479c00def62ed953c83a92.jsonld"
#response = client.get_object(bucket, path)
#jsonld = response.data

path = f"graphs/latest/iris_release.nq"
response = client.get_object(bucket, path)
nq = response.data

dg = nq

In [17]:
fna = []   # array to hold errors
try:
    g = Graph().parse(dg, format='nquads')
    #g = Graph().parse(dg, format='n3')
    result = g.serialize(format='ttl')# .decode('utf-8')
    kg.load_rdf_text(result)
except:
    print("An exception occurred with {}".format(fn))
    fna.append(fn)

conforms, report_graph, report_text = kg.validate(
    shacl_graph=sg,
    shacl_graph_format="ttl"
)

kg.load_rdf_text(data=report_graph.save_rdf_text(), format="ttl")

In [18]:
from io import BytesIO

sparql = """
SELECT  ?severity  ?constraint ?path ?message ?focus ?path ?value
  WHERE {
    ?id rdf:type shacl:ValidationResult .
    ?id shacl:focusNode ?focus .
    ?id shacl:resultMessage ?message .
    ?id shacl:resultSeverity ?severity .
    ?id shacl:sourceConstraintComponent ?constraint .
    OPTIONAL {
        ?id shacl:resultPath ?path .
    }
    OPTIONAL {
        ?id shacl:value ?value .
    }
  }
"""

pdf = kg.query_as_df(sparql)
df = pdf  #.to_pandas()  #  including .to_pandas() breaks with papermill for reasons unknown at this time if to_pandas() is used, needed in my kglab conda env

if 'severity' in df.columns:
    dfc = df.groupby('severity').count().reset_index().rename(columns={'path': 'Count'})
    ctst = pd.crosstab(df['message'], df['severity'],  margins = False , margins_name = 'Totals')

    s1 = str("Checking {} object(s)".format(len(dg) ))
    print(s1)
    print(ctst)
    f = BytesIO()
    length = f.write(bytes(ctst.to_csv(), 'utf-8'))
    f.seek(0)
    client.put_object(bucket, "reports/iris/shacl.csv", f, length)

    sns.set(rc={'figure.figsize':(11.7,8.27)})
    sns.heatmap(ctst, annot=True, fmt=".0f", cmap = sns.cm.crest)
    plt.savefig('heatmap_{}.png'.format(date_time))
else:
    print("No severity column found, all SHACL validations must have passed.")

No severity column found, all SHACL validations must have passed.
